##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 多手柄老虎机简介

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/agents/tutorials/intro_bandit"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/agents/tutorials/intro_bandit.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/agents/tutorials/intro_bandit.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/agents/tutorials/intro_bandit.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

## 仅保存权重值。通常在训练模型时使用。

多臂老虎机 (MAB) 是一种机器学习框架，其中的代理必须选择操作（臂）以最大化其长期累积奖励。在每个轮次中，代理都会收到有关当前状态（上下文）的一些信息，随后，它会根据此信息和在之前轮次中收集到的经验来选择一个操作。在每个轮次结束时，代理会收到与所选操作相关的奖励。

或许，最纯粹的示例就是把它的名字借给了 MAB 的问题：想象一下，我们面对着 `k` 台老虎机（单臂老虎机），我们需要找出哪一台具有最高的赔付，同时又不会损失太多金钱。

![Multi-Armed Bandits](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Las_Vegas_slot_machines.jpg/320px-Las_Vegas_slot_machines.jpg)

每台机器尝试一次，然后选择赔付最高的机器并不是一种良好的策略：代理可能会选择一台刚开始的结果很幸运但总体上不太理想的机器。相反，代理应反复回来选择看起来不那么好的机器，以便收集有关它们的更多信息。这是多臂老虎机中的主要挑战：代理必须在利用先验知识和探索之间找到正确的组合，以避免忽略最佳操作。

每次学习器做出决定时，更实际的 MAB 实例都会涉及一些附加信息。我们将此附加信息称为“上下文”或“观测值”。


## 多臂老虎机和强化学习

为什么 TF-Agents 库中有 MAB 套件？RL 和 MAB 之间有什么联系？可以将多手柄老虎机视为强化学习的特例。引用 [RL 简介](https://tensorflow.google.cn/agents/tutorials/0_intro_rl)：

*在每个时间步骤，代理都会根据其策略 $\pi(a_t|s_t)$（其中 $s_t$ 是来自环境的当前观测值）对环境执行操作并获得奖励 $r_{t+1}$ 和来自环境的下一个观测值 $s_{t+1}$。目标是改进政策，以使奖励总和（回报）最大化。*

在一般 RL 情况下，下一个观测值 $s_{t+1}$ 取决于先前的状态 $s_t$ 和策略采取的操作 $a_t$。最后一部分将 MAB 与 RL 分开：在 MAB 中，下一个状态（即观测值）不依赖于代理选择的操作。

这种相似性使我们可以重用 TF-Agents 中存在的所有概念。

- 一个输出观测值并用奖励来响应操作的**环境**。
- 一种基于观测值输出操作的**策略**，以及
- 一个基于先前的观测值-操作-奖励元组重复更新策略的**代理**。


## 蘑菇环境

为便于说明，我们使用一个名为“蘑菇环境”的小型示例。蘑菇数据集（[Schlimmer，1981 年](https://archive.ics.uci.edu/ml/datasets/Mushroom)）由食用蘑菇和毒蘑菇的带标签样本组成。特征包括蘑菇不同部分的形状、颜色、大小以及气味等。

![mushroom](https://archive.ics.uci.edu/ml/assets/MLimages/Large73.jpg)

与所有监督学习数据集一样，蘑菇数据集也可以转换为上下文 MAB 问题。我们使用 [Riquelme 等人（2018 年）](https://arxiv.org/pdf/1802.09127.pdf)使用的方法。在此转换中，代理接收蘑菇的特征，决定是否吃下。吃下食用蘑菇获得 +5 的奖励，而吃下毒蘑菇则会以相等的概率获得 +5 或 -35。不吃蘑菇会导致 0 奖励，而与蘑菇的类型无关。下表汇总了奖励分配：

> ```
>
> ```

```
       | edible | poisonous
```

-----------|--------|---------- 吃下  |     +5 | -35 / +5 不吃 |      0 |        0

```

```

## LinUCB 代理

在给定观测值的情况下，要在上下文老虎机环境中获得出色的表现，需要很好地评估每个操作的奖励函数。一种可能性是使用线性函数评估奖励函数。也就是说，对于每个操作 $i$，我们都尝试找到参数 $\theta_i\in\mathbb R^d$，使估算值

$r_{t, i} \sim \langle v_t, \theta_i\rangle$

尽可能接近现实。其中，$v_t\in\mathbb R^d$ 是在时间步骤 $t$ 接收到的上下文。随后，如果代理对自己的估计值非常有信心，则可以选择 $\arg\max_{1, ..., K}\langle v_t, \theta_k\rangle$ 来获取最高的预期奖励。

如上所述，简单地选择具有最佳估算奖励的手柄并不是一种很好的策略。线性 Estimator 代理中有多种将利用和探索混合的方法，其中最著名的一种是线性置信区间上界 (LinUCB) 算法（请参阅 [Li 等人 2010 年的文章](https://arxiv.org/abs/1003.0146)）。LinUCB 有两个主要构建块（省略了一些详细信息）：

1. 它使用线性最小二乘法维护每个手柄参数的估计值：$\hat\theta_i\sim X^+_i r_i$，其中 $X_i$ 和 $r_i$ 是选择手柄 $i$ 的轮次的堆叠上下文和奖励，而 $()^+$  是伪逆。
2. 对于上述估计值，它维护由逆协方差 $X_i^\top X_i$ 定义的*置信椭圆*。

LinUCB 的主要理念是“不确定行为优先探索”。代理通过将估计值增加与这些估计值的方差相对应的量来纳入探索。这就是置信椭圆生效的位置：对于每个臂，乐观估计值为 $\hat r_i = \max_{\theta\in E_i}\langle v_t，其中 $E_i$ 是绕 $\hat\theta_i$ 的椭圆。代理选择看起来最佳的臂 $\arg\max_i\hat r_i$。

当然，上面的说明只是对 LinUCB 作用的一个直观而肤浅的总结。可以在我们代码库（[此处](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/lin_ucb_agent.py)）中找到一个实现

## 后续步骤

如果您想查看关于老虎机库的更详细教程，请参阅我们的[老虎机教程](https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/bandits_tutorial.ipynb)。相反，如果您想立即开始探索我们的库，请访问[此处](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits)。如果您更希望立即开始训练，则可以查看[这里](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2)的一些端到端示例，包括上面介绍的使用 LinUCB 的蘑菇环境（[此处](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2/train_eval_mushroom.py)）。 